In [3]:
import os
os.getcwd()

'/Users/derekdewald/Documents/Python/Github_Repo/JupyterNotebooks'

In [1]:
import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

In [2]:
from Connections import DownloadFilesFromGit,BackUpGoogleSheets

DownloadFilesFromGit(output_folder='/Users/derekdewald/Documents/Python/Github_Repo/JupyterNotebooks/downloaded_py_files/')
BackUpGoogleSheets()

Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/ProcessSheet
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/DataDashboardSheet
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/CodingDDashboard
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/ML_Pipeline
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/Mapping Sheet
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/SKLearn Models
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/Notes
Counld Not Print Record 7
Counld Not Print Record 8


In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.utils import all_estimators
import mlflow
import time
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load Diabetes Dataset
diabetes = datasets.load_diabetes()
df = pd.DataFrame(diabetes['data'], columns=diabetes['feature_names'])
df['Target'] = diabetes['target']

# Function to Apply Scaling
def apply_scaling(X_train, X_test, scaler=None):
    scalers = {
        'standard': StandardScaler(),
        'normalization': MinMaxScaler()
    }
    if scaler in scalers:
        scaler_instance = scalers[scaler]
        X_train = scaler_instance.fit_transform(X_train)
        X_test = scaler_instance.transform(X_test)
    return X_train, X_test

# ML Pipeline Function
def MLPipeline(df, project_name, scaler=None, ml_model_type='regressor', target_column='Target', test_size=0.2):
    mlflow.set_experiment(project_name)

    X = df.drop(columns=[target_column])
    y = df[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_test = apply_scaling(X_train, X_test, scaler=scaler)

    model_list = all_estimators(type_filter=ml_model_type)
    results = []

    for name, model_class in model_list:
        try:
            model = model_class()
            start_time = time.time()

            with mlflow.start_run(run_name=name):
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                if ml_model_type == "classifier":
                    metric = accuracy_score(y_test, y_pred)
                    mlflow.log_metric("Accuracy", metric)
                else:
                    # Compute RMSE manually (fix for 'squared' error)
                    metric = mean_squared_error(y_test, y_pred) ** 0.5  
                    mlflow.log_metric("RMSE", metric)

                mlflow.sklearn.log_model(model, name)
                mlflow.log_param("Model", name)
                mlflow.log_param("Training Time", round(time.time() - start_time, 2))

                results.append({
                    "Model": name,
                    "Metric": metric,
                    "Time (s)": round(time.time() - start_time, 2)
                })

        except Exception as e:
            print(f"⚠️ {name} failed: {str(e)}")  # Handle model failures but continue

    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by="Metric", ascending=(ml_model_type == "regressor"))

    # Display DataFrame
    import ace_tools as tools
    tools.display_dataframe_to_user(name="ML Model Performance", dataframe=results_df)

    return results_df

sql = '''
with base as (
    select
        membernbr
        ,membername
        ,taxid
        ,memberstatus
        ,PERSDOB
    from member
    where memberstatus in ('GOOD STANDING','ACTIVE')
),

share_bal as (
     select 
        membernbr
        ,sum(case when MINOR = 'ES01' then currbal else 0 end) as CURRENT_ES01
        ,max(case when daysdelinq >=90 then 1 else 0 end) as CURRENT_DELINQ
        from account
        group by membernbr
),

dec_del as (
    select distinct 
        membernbr, 
        1 as DELIQUENT_DEC2024
    from accounthist 
    where effdate = '30-Nov-24' and daysdelinq>=90 and major !='LEAS'
),

dec_shar_bal as (
    select 
        t2.membernbr,
        sum(t1.notebal) as BAL_DEC13
    from acctbal t1
    left join account t2 
    on t1.acctnbr=t2.acctnbr
    where t1.effdate='13-Dec-24' and t1.minor='ES01'
    group by t2.membernbr
 ),
 
 feb_shar_bal as (
    select 
        t2.membernbr,
        sum(t1.notebal) as BAL_FEB13
    from acctbal t1
    left join account t2 
    on t1.acctnbr=t2.acctnbr
    where t1.effdate='13-Feb-25' and t1.minor='ES01'
    group by t2.membernbr
 )
 
select t1.*, t2.CURRENT_ES01,t2.CURRENT_DELINQ,t3.DELIQUENT_DEC2024,t4.bal_dec13,t5.bal_feb13
from base t1
left join share_bal t2 on t1.membernbr=t2.membernbr
left join dec_del   t3 on t1.membernbr=t3.membernbr
left join dec_shar_bal t4 on t1.membernbr=t4.membernbr
left join feb_shar_bal t5 on t1.membernbr=t5.membernbr

'''

import numpy as np
import pandas as pd

def calculate_trends(data, id_column, time_columns):
    """
    Calculate trends (slopes) for entities over time using linear regression.
    
    Parameters:
        data (pd.DataFrame): Input DataFrame containing entity IDs and time-based columns.
        id_column (str): Column name for the unique entity identifier (e.g., member_id).
        time_columns (list of str): List of column names representing the time periods (e.g., months).
    
    Returns:
        pd.DataFrame: DataFrame containing entity IDs, slopes, and trends.
    """
    # Extract unique entity IDs and the time-series data
    entity_ids = data[id_column].unique()
    time_data = data[time_columns].values
    
    # Prepare the X (time) matrix
    time_indices = np.arange(1, len(time_columns) + 1)  # Generate time indices
    X = np.vstack([time_indices, np.ones(len(time_indices))]).T  # Add a bias term for the intercept

    # Perform linear regression for all entities
    slopes = np.linalg.lstsq(X, time_data.T, rcond=None)[0][0]  # Extract slopes
    
    # Prepare results DataFrame
    results = pd.DataFrame({
        id_column: entity_ids,
        "slope": slopes
    })
    
    # Classify trends
    results["trend"] = np.where(
        results["slope"] > 0, "increasing",
        np.where(results["slope"] < 0, "decreasing", "stable")
    )
    
    return results


test_dict = {}

final_df = pd.DataFrame()

for model in models:
    for metric in ['F1','AUC','Accuracy']:   
        test_dict.setdefault('Top 5 Performing Models',{})[metric] = np.mean(value_list[:4])
        test_dict.setdefault('Top Performing Model',{})[metric] = value_list[0]
        test_dict.setdefault('All Models Mean',{})[metric] = np.mean(value_list)
        
        temp_df = pd.DataFrame(test_dict).T.reset_index().rename(columns={'index':'Type'})
        temp_df['Observed Value'] = model
    
    final_df = pd.concat([final_df,temp_df])


import itertools

list_1 = [1,2,3,4]
list_2 = [5,6,7,8]
list_3 = ['d','e','f','g']
list_4 = ['&','*',')','(']

combinations = list(itertools.product(list_1,list_2,list_3,list_4))
combinations[0:10]


import os 
os.startfile(file_location)


import bcdata
bcevacdata = bcdata.get_data('evacuation-orders-and-alerts', as_gdf=True)


from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,roc_curve,roc_auc_score,mean_squared_error
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb


from gffg.DF_PROCESSING import brackets_v3

from itertools import product
import pandas as pd
import numpy as np
import timeit

# Define Macro Variables
random_state = 42
rf_nodes = 100
xgb_objective='binary:logistic'
xgb_eval_metric='logloss'

ml_model_dict = {
    'Linear Regression': {
        'Type': 'Linear',
        'Description': 'A linear approach to modeling the relationship between a dependent variable and one or more independent variables.',
        'UseCase': "Regression",
        'ModelStrengths': "Simple, interpretable, computationally efficient, works well for linearly separable data.",
        'Model Weaknesses': "Assumes linear relationships, sensitive to outliers.",
        'lowerbound_feature_threshold':.1,
        'upperbound_feature_threshold':.5,
        'ReferenceWorkbooks': "",
        'SKlearn': LinearRegression()
    },
    'Logistic Regression': {
        'Type': 'Linear',
        'Description': 'A regression model that predicts probabilities using the sigmoid function for binary classification problems.',
        'UseCase': ["Classification"],
        'ModelStrengths': "Simple, interpretable, performs well for linearly separable data.",
        'Model Weaknesses': "Struggles with non-linear relationships, assumes linear relationship between features and log-odds.",
        'ReferenceWorkbooks': "",
        'lowerbound_feature_threshold':.1,
        'upperbound_feature_threshold':.5,
        'SKlearn': LogisticRegression(random_state=random_state,class_weight=None)
    },
    'Decision Tree': {
        'Type': 'Tree',
        'Description': 'A model that splits data into branches to make predictions based on feature values.',
        'UseCase': ["Classification", "Regression"],
        'ModelStrengths': "Easy to interpret, handles both numerical and categorical data.",
        'Model Weaknesses': "Prone to overfitting, unstable with small changes in data.",
        'lowerbound_feature_threshold':.05,
        'upperbound_feature_threshold':.1,
        'ReferenceWorkbooks': "",
        'SKlearn': DecisionTreeClassifier(random_state=random_state)
    },
    'Random Forest': {
        'Type': 'Ensemble',
        'Description': 'An ensemble of decision trees using bagging to reduce overfitting and improve accuracy.',
        'UseCase': ["Classification", "Regression"],
        'ModelStrengths': "Reduces overfitting, robust to noise, handles high-dimensional data well.",
        'Model Weaknesses': "Computationally expensive, less interpretable than a single tree.",
        'lowerbound_feature_threshold':.05,
        'upperbound_feature_threshold':.1,
        'ReferenceWorkbooks': "",
        'Parameters':{'n_estimators':'Total Number of Decision Trees which are created by the model. The more trees the more likely to overfit and longer to process, the fewer decision nodes, the less unique information is captured',
                     'class_weight':'Model Automatically Adjusts the weight of each sample, attempting to adjust for the pertinent data imbalance. It does not undersample or remove.'},
        'SKlearn': RandomForestClassifier(random_state=random_state, n_estimators=rf_nodes,class_weight=None)
    },
    'Extra Tree': {
        'Type': 'Ensemble',
        'Description': 'Similar to Random Forest but selects splits randomly rather than optimizing them.',
        'UseCase': ["Classification", "Regression"],
        'ModelStrengths': "Reduces overfitting further, computationally efficient compared to Random Forest.",
        'Model Weaknesses': "Slightly less accurate in some cases, still computationally heavy.",
        'lowerbound_feature_threshold':.05,
        'upperbound_feature_threshold':.1,
        'ReferenceWorkbooks': "",
        'SKlearn': ExtraTreesClassifier(random_state=random_state)
    },
    'Gradient Boosting': {
        'Type': 'Ensemble',
        'Description': 'Builds models sequentially, correcting errors from previous models in an additive fashion.',
        'UseCase': ["Classification", "Regression"],
        'ModelStrengths': "Highly accurate, handles non-linear relationships well.",
        'Model Weaknesses': "Sensitive to hyperparameters, prone to overfitting on noisy data.",
        'ReferenceWorkbooks': "",
        'lowerbound_feature_threshold':.05,
        'upperbound_feature_threshold':.1,
        'SKlearn': GradientBoostingClassifier(random_state=random_state)
    },
    'XGBoost': {
        'Type': 'Ensemble',
        'Description': 'An optimized gradient boosting model with regularization to prevent overfitting.',
        'UseCase': ["Classification", "Regression"],
        'ModelStrengths': "Fast, scalable, handles sparse data, regularization prevents overfitting.",
        'Model Weaknesses': "Requires careful tuning of hyperparameters, computationally expensive.",
        'lowerbound_feature_threshold':.05,
        'upperbound_feature_threshold':.1,
        'ReferenceWorkbooks': "",
        'SKlearn': xgb.XGBClassifier(objective=xgb_objective, eval_metric=xgb_eval_metric, use_label_encoder=False, random_state=random_state)
    },
    'Ada': {
        'Type': 'Ensemble',
        'Description': 'Boosting technique that combines multiple weak learners (usually decision stumps) into a strong learner.',
        'UseCase': ["Classification", "Regression"],
        'ModelStrengths': "Handles weak learners well, reduces overfitting.",
        'Model Weaknesses': "Sensitive to noisy data, requires careful tuning of learning rate.",
        'ReferenceWorkbooks': "",
        'lowerbound_feature_threshold':.05,
        'upperbound_feature_threshold':.1,
        'SKlearn': AdaBoostClassifier(random_state=random_state)
    },
    'Support Vector Machine (SVM)': {
        'Type': 'Kernel-Based',
        'Description': 'A model that finds the optimal hyperplane for separating classes.',
        'UseCase': ["Classification", "Regression"],
        'ModelStrengths': "Effective in high-dimensional spaces, works well with clear margins of separation.",
        'Model Weaknesses': "Computationally expensive, struggles with large datasets.",
        'ReferenceWorkbooks': "",
        'SKlearn': None
    },
    'K-Nearest Neighbors (KNN)': {
        'Type': 'Instance-Based',
        'Description': 'A model that classifies points based on their proximity to neighbors.',
        'UseCase': ["Classification", "Regression"],
        'ModelStrengths': "Simple to implement, works well with small datasets.",
        'Model Weaknesses': "Computationally expensive with large datasets, sensitive to irrelevant features.",
        'ReferenceWorkbooks': "",
        'SKlearn': None
    }
}

ml_dict_df = pd.DataFrame(ml_model_dict).T.reset_index().rename(columns={'index':'Model'})

scaler_dict = {
    'Standard':StandardScaler(),
    'Min Max': MinMaxScaler(),
    'Robust':RobustScaler()}


def BalanceTargetDistribution(df,
                              Target,
                              desired_percentage,
                              TargetValue=1,):
    
    df = df.copy()

    df0 = df[df[Target]==TargetValue].copy()
    df1 = df[df[Target]!=TargetValue].copy()
    
    if (desired_percentage>0)&(desired_percentage<1):
        req_columns = int(round(len(df0)/desired_percentage,0))
        
        return pd.concat([df0,df1.sample(req_columns)]).sample(frac=1).reset_index(drop=True)
        
    else:
        print('Desired Percentage Outside of Allowable Range (0-1), Please Select a New Value')


def ClassificationMetrics(df,
                          model="",
                          scaler="",
                          balance="",
                          sequence_num=0,
                          prediction='PREDICTION',
                          actual='ACTUAL',
                          new_column_name='RESULT'):
    '''
    
    Accuracy: Number of Correct Preditions. Can be misleading with Imbalanced data, where it simply predicts Majority Class
    
    Precision: Number of Positive Correctly Predicted as a Percentage of Total Positive Instances. Can be misleading when the total number of predictions is very high (IE Predict EVerything Positive, this number will be 100%, but will waste so much time because of false predictions)
    
    Recall: The Percentage of Correct Predictions as a Percentage of Total Positive Predictions. How Accurate your prediction is. This number can be misleading because you can have a High Recall by simply predicting things only with a high degress of certainty, missing out on fringe cases.
    
    F1: Harmonic Mean when Balancing Precision and Recall
    
    '''
    
    results_dict = {}
    
    
    results_dict['SequenceNumber'] = sequence_num

    print(scaler)
    
    if len(model)!=0:
        results_dict['Model'] = model
    if isinstance(scaler,str):
        results_dict['Scaler'] = scaler 
    if len(balance)!=0:
        results_dict['Training Set Balance Perc'] = balance
    
    condition = [(df[prediction]==1)&(df[actual]==df[prediction]),
                 (df[prediction]==0)&(df[actual]==df[prediction]),
                 (df[prediction]==1)&(df[actual]!=df[prediction]),
                 (df[prediction]==0)&(df[actual]!=df[prediction])]
    
    values = ['True Positives','True Negatives','False Positives (I)','False Negatives (II)']
    
    df[new_column_name] = np.select(condition,values)
    
    results_dict['True Positives'] =       len(df[df['RESULT']=='True Positives'])
    results_dict['True Negatives'] =       len(df[df['RESULT']=='True Negatives'])
    results_dict['False Positives (I)'] =  len(df[df['RESULT']=='False Positives (I)'])
    results_dict['False Negatives (II)'] = len(df[df['RESULT']=='False Negatives (II)'])
    
    
    results_dict['Total Records'] = len(df)
    results_dict['Correct Predictions'] =   len(df[df['RESULT'].isin(['True Negatives','True Positives'])])
    results_dict['Incorrect Predictions'] = len(df[df['RESULT'].isin(['False Negatives (II)','False Positives (I)'])])
    results_dict['Actual Positives'] = len(df[df['RESULT'].isin(['False Negatives (II)','True Positives'])])
    results_dict['Actual Negatives'] = len(df[df['RESULT'].isin(['False Positives (I)','True Negatives'])])
    
    try:
        results_dict['Recall'] = results_dict['True Positives']/(results_dict['True Positives']+results_dict['False Negatives (II)'])     # How Many Positives were Actually Predicted
        results_dict['Precision']    = results_dict['True Positives']/(results_dict['True Positives']+results_dict['False Positives (I)']) # How Many Predictions were Correct
        results_dict['F1'] = 2*(results_dict['Precision']*results_dict['Recall'])/(results_dict['Precision']+results_dict['Recall'])  # Harmonic Mean 
        
    except:
        results_dict['Recall'] = 0
        results_dict['Precision'] = 0
        results_dict['F1'] = 0
        
    results_dict['Accuracy']  = results_dict['Correct Predictions']/results_dict['Total Records']     # How Correct your Model was overall
    results_dict['AUC']       = roc_auc_score(df[actual],df[prediction])
    
    return df,pd.DataFrame([results_dict.values()],columns=results_dict.keys())

def GenerateMLModels(df,
                     Features,
                     Target,
                     model_list,
                     scaler_list,
                     balance_list,
                     log_max_it=1000,
                     rf_estimators=50,
                     include_narrative=0):
    
    model_dict = {}
    final_result_df = pd.DataFrame()
    
    param_list = list(product(model_list,scaler_list,balance_list))
    
    for count,params in enumerate(param_list):
        print(count,params)
        
        model = params[0]
        scaler = params[1]
        if params[2]!=0:
            balance_target_observations = [1,params[2]]
        else:
            balance_target_observations = [0,0]
        
        run_dict,result = MLManualPipeline(df=df,
                                           Features=Features,
                                           Target=Target,
                                           model=model,
                                           scaler_=scaler,
                                           balance_target_observations=balance_target_observations,
                                           include_narrative=include_narrative,
                                           rf_estimators=rf_estimators,
                                           SequenceNumber=count,
                                           log_max_it=log_max_it)
        
        model_dict[count] = run_dict
        final_result_df = pd.concat([final_result_df,result])
        
    return model_dict,final_result_df




def ModelPerformanceByCategory(df,
                               primary_key='SequenceNumber',
                               ml_parameters=['Model','Scaler','Training Set Balance Perc'],
                               observations=20,
                               metrics=['AUC','F1','Accuracy']):
    
    # Create List of Models Based on Original Input Dataframe
    models = list(df['Model'].unique())
    
    # Create Values for Particular Metric under review
    for metric in metrics:
        # Sort Value based on Metric
        metric_df = df.sort_values(metric,ascending=False)
        # Iterate through Categories as defined by Function
        temp = pd.DataFrame()
        for ml_ in ml_parameters:
            _ = pd.DataFrame(metric_df[ml_].fillna("").head(observations).value_counts()).reset_index().rename(columns={'index':'Observed Value',ml_:metric})
            _['Type'] = ml_
            
            temp = pd.concat([temp,_])
            
        # Generate Metric Summary Value 
        # Create Segments
        brackets_v3(metric_df,metric,f"{metric}_segment",[0,.25,.5,.6,.75,.9])
        perc = pd.DataFrame(metric_df[f"{metric}_segment"].value_counts()).reset_index().rename(columns={'index':"Observed Value",f"{metric}_segment":metric})
        perc['Type'] = 'Model Performance'
        
        temp = pd.concat([temp,perc[perc[metric]!=0]])
        
        try:
            final_df = final_df.merge(temp,on=['Observed Value','Type'],how='outer')
        except:
            final_df = temp.copy()
            
        test_dict = {}
        perf_df = pd.DataFrame()
        for model in models:
            for metric in ['F1','AUC','Accuracy']:
        
                value_list = df[df['Model']==model].sort_values(metric,ascending=False)[metric].tolist()
                test_dict.setdefault('Model Best Five',{})[metric] = np.mean(value_list[:4])
                test_dict.setdefault('Model Best Performing',{})[metric] = value_list[0]
                test_dict.setdefault('Model Mean Performance',{})[metric] = np.mean(value_list)

                temp_df = pd.DataFrame(test_dict).T.reset_index().rename(columns={'index':'Type'})
                temp_df['Observed Value'] = model
            
            perf_df = pd.concat([perf_df,temp_df])

    return pd.concat([final_df,perf_df]).sort_values(['Type',metrics[0]],ascending=[True,False]).set_index(['Observed Value','Type']).fillna(0)
    


def MLManualPipeline(df,
                     Features,
                     Target,
                     model,
                     balance_target_observations=[0,.25],
                     scaler_=None,
                     test_size=.3,
                     random_state=42,
                     include_narrative=1,
                     rf_estimators=50,
                     log_max_it=100,
                     SequenceNumber=0,
                     time_model=1,
                     xgb_objective='binary:logistic',
                     xgb_eval_metric='logloss'):
    
    start_time = timeit.default_timer()
        
    generated_models = {}
    
    # Create Copy to ensure original data is not altered
    df = df.copy()

    if balance_target_observations[0]==1:
        df = BalanceTargetDistribution(df=df,
                                       Target=Target,
                                       desired_percentage=balance_target_observations[1])
        balance_target_observations = f'{balance_target_observations[1]}'
    else:
        balance_target_observations = '0'
        
    # If features not specified, entire Dataset to be processed (less Target), otherwise include only features
    if len(Features) == 0:
        Features = list(df.drop(Target,axis=1).columns)
        X = np.array(df[Features].copy())
        df_len = len(df)
        feature_count = len(X[0])
    else:
        X = np.array(df[Features])
        df_len = len(df)
        feature_count = len(X[0])
        
    # Create Target Array - Target already provided as a List
    y = np.array(df[Target].squeeze())
    target_vc = df[Target].value_counts()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    if scaler_ in scaler_dict.keys():
        scaler = scaler_dict[scaler_]
        X_train = scaler.fit_transform(X_train)
        x_test = scaler.transform(X_test)
    else:
        scaler = None
        
    if include_narrative==1:
        print(f"Total Number of Records in Dataset: {df_len}\ntotal features: {feature_count}\nTarget Distribution:\n{target_vc}\n")
        print(f"Training Data Set:\nTraining Features:{len(X_train)}, Training Target:{len(y_train)}\nTraining Target Distribution:\n{pd.DataFrame(y_train).value_counts()}\n")
        print(f"Testing Data Set:\nTest Features:{len(X_test)}, Test Target:{len(y_test)}Training Target Distribution:\n{pd.DataFrame(y_test).value_counts()}\n")
        
    if model=='Logistic Regression':
        
        # Create Model
        logreg=LogisticRegression(max_iter=log_max_it)
        
        # Train Model
        logreg.fit(X_train, y_train)
        
        # Predict on Text
        y_pred = logreg.predict(X_test)
        
        # Generate Test DataFrame
        result_df = pd.concat([pd.DataFrame(y_pred,columns=['PREDICTION']),pd.DataFrame(y_test,columns=['ACTUAL'])],axis=1)
        
        # Create Feature Importance DataFrame
        feature_importance = pd.DataFrame({'FEATURE':Features,
                                           'IMPORTANCE':np.abs(logreg.coef_[0]),}).sort_values(by='IMPORTANCE',ascending=False)
        
        # Generate Results and Dataframe
        result_df, performance = ClassificationMetrics(df=result_df,
                                                       model=model,
                                                       scaler=scaler_,
                                                       balance=balance_target_observations,
                                                       sequence_num=SequenceNumber)
        stop_time = timeit.default_timer() - start_time
        
        performance['RunTime'] = stop_time
        generated_models[SequenceNumber] = {'ModelType':model,
                                            'Model':logreg,
                                            'Scaler':scaler,
                                            'FeatureImportance':feature_importance,
                                            'BinaryResultDF':result_df,
                                            'ModelPerformance':performance}
        
        try:
            # Generate a Balanced Variant of the Logistic Regression Where 
            perc_dist = len(df[df[Target]==0])/len(df)
            if perc_dist>.6:
                lr0=LogisticRegression(max_iter=log_max_it,class_weight='balanced')
                lr0.fit(X_train, y_train)
                y_pred_rf = lr0.predict(X_test)
                result_df = pd.concat([pd.DataFrame(y_pred_rf,columns=['PREDICTION']),
                                       pd.DataFrame(y_test,columns=['ACTUAL'])],axis=1)
                feature_importance = pd.DataFrame({'FEATURE':Features,
                                                   'IMPORTANCE':rf.feature_importances_}).sort_values(by='IMPORTANCE',ascending=False)
                result_df, performance = ClassificationMetrics(df=result_df,
                                                               model='Logistic Regression - Balanced',
                                                               scaler=scaler_,
                                                               balance=balance_target_observations,
                                                               sequence_num=SequenceNumber+1000000)
                stop_time = timeit.default_timer() - start_time
                performance['RunTime'] = stop_time
                generated_models[SequenceNumber+1000000] = {'ModelType':'Logistic Regression - Balanced',
                                                        'Model':lr0,
                                                        'Scaler':scaler,
                                                        'FeatureImportance':feature_importance,
                                                        'BinaryResultDF':result_df,
                                                        'ModelPerformance':performance}
        except:
            print('Attempted to Review Random Forrest - Balanced, could not')

    elif model =='Decision Tree':    
        ############################################ ESTIMATORS
        dt = DecisionTreeClassifier(random_state=random_state)
        dt.fit(X_train, y_train)
        y_pred_dt = dt.predict(X_test)
        result_df = pd.concat([pd.DataFrame(y_pred_dt,columns=['PREDICTION']),pd.DataFrame(y_test,columns=['ACTUAL'])],axis=1)
        feature_importance = pd.DataFrame({'FEATURE':Features,
                                           'IMPORTANCE':dt.feature_importances_}).sort_values(by='IMPORTANCE',ascending=False)
        result_df, performance = ClassificationMetrics(df=result_df,
                                                       model=model,
                                                       scaler=scaler_,
                                                       balance=balance_target_observations,
                                                       sequence_num=SequenceNumber)
        stop_time = timeit.default_timer() - start_time
        performance['RunTime'] = stop_time
        
        
        generated_models[SequenceNumber] = {'ModelType':model,
                                            'Model':dt,
                                            'Scaler':scaler,
                                            'FeatureImportance':feature_importance,
                                            'BinaryResultDF':result_df,
                                            'ModelPerformance':performance}

    elif model =='Random Forest':    
        ############################################ ESTIMATORS
        rf = RandomForestClassifier(random_state=random_state, n_estimators=rf_estimators)
        rf.fit(X_train, y_train)
        y_pred_rf = rf.predict(X_test)
        result_df = pd.concat([pd.DataFrame(y_pred_rf,columns=['PREDICTION']),
                               pd.DataFrame(y_test,columns=['ACTUAL'])],axis=1)
        feature_importance = pd.DataFrame({'FEATURE':Features,
                                           'IMPORTANCE':rf.feature_importances_}).sort_values(by='IMPORTANCE',ascending=False)
        result_df, performance = ClassificationMetrics(df=result_df,
                                                       model=model,
                                                       scaler=scaler_,
                                                       balance=balance_target_observations,
                                                       sequence_num=SequenceNumber)
        stop_time = timeit.default_timer() - start_time
        performance['RunTime'] = stop_time
        generated_models[SequenceNumber] = {'ModelType':model,
                                            'Model':rf,
                                            'Scaler':scaler,
                                            'FeatureImportance':feature_importance,
                                            'BinaryResultDF':result_df,
                                            'ModelPerformance':performance}
        
        try:
            # Generate a Balanced Variant of the Random Forest Where 
            perc_dist = len(df[df[Target]==0])/len(df)
            if perc_dist>.6:
                rf0 = RandomForestClassifier(random_state=random_state, n_estimators=rf_estimators,class_weight='balanced')
                rf0.fit(X_train, y_train)
                y_pred_rf = rf0.predict(X_test)
                result_df = pd.concat([pd.DataFrame(y_pred_rf,columns=['PREDICTION']),
                                       pd.DataFrame(y_test,columns=['ACTUAL'])],axis=1)
                feature_importance = pd.DataFrame({'FEATURE':Features,
                                                   'IMPORTANCE':rf.feature_importances_}).sort_values(by='IMPORTANCE',ascending=False)
                result_df, performance = ClassificationMetrics(df=result_df,
                                                               model='Random Forest - Balanced',
                                                               scaler=scaler_,
                                                               balance=balance_target_observations,
                                                               sequence_num=SequenceNumber+1000000)
                stop_time = timeit.default_timer() - start_time
                performance['RunTime'] = stop_time
                generated_models[SequenceNumber+1000000] = {'ModelType':'Random Forest - Balanced',
                                                        'Model':rf0,
                                                        'Scaler':scaler,
                                                        'FeatureImportance':feature_importance,
                                                        'BinaryResultDF':result_df,
                                                        'ModelPerformance':performance}
        except:
            print('Attempted to Review Random Forrest - Balanced, could not')
                
    elif model == 'Gradient Boosting':
        gb = GradientBoostingClassifier()
        gb.fit(X_train, y_train)
        y_pred_gb = gb.predict(X_test)
        result_df = pd.concat([pd.DataFrame(y_pred_gb,columns=['PREDICTION']),pd.DataFrame(y_test,columns=['ACTUAL'])],axis=1)
        feature_importance = pd.DataFrame({'FEATURE':Features,
                                           'IMPORTANCE':gb.feature_importances_}).sort_values(by='IMPORTANCE',ascending=False)
        
        result_df, performance = ClassificationMetrics(df=result_df,
                                                       model=model,
                                                       scaler=scaler_,
                                                       balance=balance_target_observations,
                                                       sequence_num=SequenceNumber)
        stop_time = timeit.default_timer() - start_time
        performance['RunTime'] = stop_time
        generated_models[SequenceNumber] = {'ModelType':model,
                                            'Model':gb,
                                            'Scaler':scaler,
                                            'FeatureImportance':feature_importance,
                                            'BinaryResultDF':result_df,
                                            'ModelPerformance':performance}
    elif model == 'Ada':
        ada = AdaBoostClassifier()
        ada.fit(X_train, y_train)
        y_pred_ada = ada.predict(X_test)
        result_df = pd.concat([pd.DataFrame(y_pred_ada,columns=['PREDICTION']),pd.DataFrame(y_test,columns=['ACTUAL'])],axis=1)
        feature_importance = pd.DataFrame({'FEATURE':Features,
                                           'IMPORTANCE':ada.feature_importances_}).sort_values(by='IMPORTANCE',ascending=False)
        
        result_df, performance = ClassificationMetrics(df=result_df,
                                                       model=model,
                                                       scaler=scaler_,
                                                       balance=balance_target_observations,
                                                       sequence_num=SequenceNumber)
        stop_time = timeit.default_timer() - start_time
        performance['RunTime'] = stop_time
        generated_models[SequenceNumber] = {'ModelType':model,
                                            'Model':ada,
                                            'Scaler':scaler,
                                            'FeatureImportance':feature_importance,
                                            'BinaryResultDF':result_df,
                                            'ModelPerformance':performance}
        
    elif model == 'Extra Tree':
        et = ExtraTreesClassifier()
        et.fit(X_train, y_train)
        y_pred_et = et.predict(X_test)
        result_df = pd.concat([pd.DataFrame(y_pred_et,columns=['PREDICTION']),pd.DataFrame(y_test,columns=['ACTUAL'])],axis=1)
        feature_importance = pd.DataFrame({'FEATURE':Features,
                                           'IMPORTANCE':et.feature_importances_}).sort_values(by='IMPORTANCE',ascending=False)
        
        result_df, performance = ClassificationMetrics(df=result_df,
                                                       model=model,
                                                       scaler=scaler_,
                                                       balance=balance_target_observations,
                                                       sequence_num=SequenceNumber)
        stop_time = timeit.default_timer() - start_time
        performance['RunTime'] = stop_time
        generated_models[SequenceNumber] = {'ModelType':model,
                                            'Model':et,
                                            'Scaler':scaler,
                                            'FeatureImportance':feature_importance,
                                            'BinaryResultDF':result_df,
                                            'ModelPerformance':performance}
        
    elif model == 'XGBoost':
        xgb_ = xgb.XGBClassifier(objective=xgb_objective,
                                eval_metric=xgb_eval_metric,
                                use_label_encoder=False,
                                random_state=random_state)
        xgb_.fit(X_train, y_train)
        y_pred_et = xgb_.predict(X_test)
        result_df = pd.concat([pd.DataFrame(y_pred_et,columns=['PREDICTION']),
                               pd.DataFrame(y_test,columns=['ACTUAL'])],axis=1)
        feature_importance = pd.DataFrame({'FEATURE':Features,
                                           'IMPORTANCE':xgb_.feature_importances_}).sort_values(by='IMPORTANCE',ascending=False)
        
        result_df, performance = ClassificationMetrics(df=result_df,
                                                       model=model,
                                                       scaler=scaler_,
                                                       balance=balance_target_observations,
                                                       sequence_num=SequenceNumber)
        stop_time = timeit.default_timer() - start_time
        performance['RunTime'] = stop_time
        
        generated_models[SequenceNumber] = {'ModelType':model,
                                            'Model':xgb,
                                            'Scaler':scaler,
                                            'FeatureImportance':feature_importance,
                                            'BinaryResultDF':result_df,
                                            'ModelPerformance':performance}    
    else:
        print('Model Not Defined, please update function')
        
    performance_metrics = pd.DataFrame()
    for key in generated_models.keys():
        performance_metrics = pd.concat([performance_metrics,generated_models[key]['ModelPerformance']])
        
    return generated_models,performance_metrics

Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/ProcessSheet
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/DataDashboardSheet
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/CodingDDashboard
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/ML_Pipeline
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/Mapping Sheet
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/SKLearn Models
Back Up Saved, /Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/Notes
Counld Not Print Record 7
Counld Not Print Record 8


In [ ]:
from SharedFolder import DuplicateFileorFolder
# Create Backup of Github_Repo
DuplicateFileorFolder('/Users/derekdewald/Documents/Python/Github_Repo/',
                      '/Users/derekdewald/Documents/GitHub_Repo_BACKUP/')

In [ ]:
def create_balanced_dataset(X,y,observations=0,column_name='BANKRUPTCY_FLAG'):
    
    '''
    Function to take observations and labels, combine them together and then select a even numher of random examples
    
    X - X_Test or X_Training
    y - y_test or y_training
    observation - Number of records from both Binary On and Binary Off Column
    column_name - Name of Binary Column to filter
    
    
    Used Default Value for purposes of reducing typing in code and given function created for Project Exclusively.
    
    '''
    
    # If length of observations is not defined, create a even 50/ 50 dataset
    if observations == 0:
        observations = y[column_name].sum()
        
    if observations>y[column_name].sum():
        observations = y[column_name].sum()
        
    temp_df = pd.concat([X,y],axis=1).copy()
    
    df1 = temp_df[temp_df[column_name]==1].sample(observations).copy()
    df2 = temp_df[temp_df[column_name]==0].sample(observations).copy()
    
    final_df = pd.concat([df1,df2])
    final_df = final_df.sample(frac=1)
    
    X = final_df.drop(column_name,axis=1)
    y = final_df[[column_name]]
    
    
def train_neural_network(X,
                y,
                input_dim,
                metrics,
                hidden_layer_sizes,
                activation, 
                optimizer,
                learning_rate,
                batch_size,
                num_epochs,
                validation_split,
                verbose=0):
                       

    # Build the model.
    model = build_binary_classification_model(input_dim=input_dim,
                                              hidden_layer_sizes=hidden_layer_sizes,
                                              activation=activation, 
                                              optimizer=optimizer,
                                              learning_rate=learning_rate,
                                              metrics=metrics)
    
    print(model.summary())     
                        
    # Train the model.
    history = model.fit(x=X,
                        y=y,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_split=validation_split,
                        verbose=verbose)

    # Retrieve the training metrics (after each train epoch) and the final test
    # accuracy.
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    plt.figure(figsize=(15,5))
    plt.plot(train_accuracy, label='train_accuracy')
    plt.plot(val_accuracy, label='validation accuracy')
    plt.xticks(range(num_epochs))
    plt.xlabel('Train epochs')
    plt.ylim(0, 1)
    plt.legend()
    plt.show()
    
 
    return history,model

    
def neural_network(X_df,
                   y_df,
                   input_dim, 
                   hidden_layer_sizes,
                   activation,
                   optimizer,
                   learning_rate,
                   metrics,
                   verbose=0):

    """Build a binary classification model using Keras.

      Args:
        input_dim: Number of features in the input data.
        hidden_layer_sizes: A list with the number of units in each hidden layer.
        activation: The activation function to use for the hidden layers.
        optimizer: The optimizer
        learning_rate: The desired learning rate for the optimizer.

      Returns:
        model: A tf.keras model.
    """
    # Instantiate Model
    model = keras.models.Sequential()

    # Add Input Layer
    model.add(layers.InputLayer(input_shape=(input_dim,)))

    # Add Hidden Layers
    for nodes in hidden_layer_sizes:
        model.add(layers.Dense(units=nodes, activation=activation))

    # Add Output Layer
    model.add(layers.Dense(units=1, activation='sigmoid'))

    # Configure optimizer and compile the model
    if optimizer == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        optimizer = keras.optimizers.legacy.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)

    return model
